In [2]:
!kubectl create secret --help

Create a secret with specified type.

 A docker-registry type secret is for accessing a container registry.

 A generic type secret indicate an Opaque secret type.

 A tls type secret holds TLS certificate and its associated key.

Available Commands:
  docker-registry   Create a secret for use with a Docker registry
  generic           Create a secret from a local file, directory, or literal value
  tls               Create a TLS secret

Usage:
  kubectl create secret (docker-registry | generic | tls) [options]

Use "kubectl create secret <command> --help" for more information about a given command.
Use "kubectl options" for a list of global command-line options (applies to all commands).


In [3]:
!kubectl create secret generic --help

Create a secret based on a file, directory, or specified literal value.

 A single secret may package one or more key/value pairs.

 When creating a secret based on a file, the key will default to the basename of the file, and the value will default to the file content. If the basename is an invalid key or you wish to chose your own, you may specify an alternate key.

 When creating a secret based on a directory, each file whose basename is a valid key in the directory will be packaged into the secret. Any directory entries except regular files are ignored (e.g. subdirectories, symlinks, devices, pipes, etc).

Examples:
  # Create a new secret named my-secret with keys for each file in folder bar
  kubectl create secret generic my-secret --from-file=path/to/bar
  
  # Create a new secret named my-secret with specified keys instead of names on disk
  kubectl create secret generic my-secret --from-file=ssh-privatekey=path/to/id_rsa --from-file=ssh-publickey=path/to/id_rsa.pub
  
  # Crea

In [6]:
!kubectl create secret generic db-creds --from-literal=username=db_user --from-literal=password=db_pass

secret/db-creds created


In [7]:
!kubectl get secret

NAME       TYPE     DATA   AGE
db-creds   Opaque   2      8s


In [8]:
!kubectl describe secret db-creds

Name:         db-creds
Namespace:    default
Labels:       <none>
Annotations:  <none>

Type:  Opaque

Data
====
password:  7 bytes
username:  7 bytes


In [9]:
!kubectl get secret db-creds -o yaml

apiVersion: v1
data:
  password: ZGJfcGFzcw==
  username: ZGJfdXNlcg==
kind: Secret
metadata:
  creationTimestamp: "2025-08-31T18:01:52Z"
  name: db-creds
  namespace: default
  resourceVersion: "101755"
  uid: dba9b23e-9311-4f30-be57-ea5ee82742bc
type: Opaque


In [11]:
!echo "ZGJfcGFzcw==" | base64 -d

'base64' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!kubectl apply -f .

pod/busybox created


In [13]:
!kubectl get pods

NAME      READY   STATUS    RESTARTS   AGE
busybox   1/1     Running   0          7s


In [14]:
!kubectl logs busybox   

db_user
db_pass


In [18]:
!kubectl apply -f .

pod/busybox created


In [19]:
!kubectl logs busybox   

db_user
db_pass


# mounting secrets as volumes

In [20]:
!kubectl get pods

NAME      READY   STATUS    RESTARTS   AGE
busybox   1/1     Running   0          3m7s


In [22]:
!kubectl delete pods --force busybox

pod "busybox" force deleted


In [23]:
!kubectl apply -f .

pod/busybox created


In [26]:
!kubectl exec busybox -- sh -c "ls /etc/db"

password
username


In [27]:
!kubectl exec busybox -- sh -c "cat /etc/db/password"

db_pass


In [28]:
!kubectl exec busybox -- sh -c "cat /etc/db/username"

db_user


In [29]:
!kubectl get pod busybox -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"labels":{"app":"busybox"},"name":"busybox","namespace":"default"},"spec":{"containers":[{"command":["sh","-c","sleep 1800"],"image":"busybox:1.36.1","name":"busybox","resources":{"limits":{"cpu":"500m","memory":"128Mi"}},"volumeMounts":[{"mountPath":"/etc/db","name":"db-secrets"}]}],"volumes":[{"name":"db-secrets","secret":{"secretName":"db-creds"}}]}}
  creationTimestamp: "2025-08-31T18:17:00Z"
  labels:
    app: busybox
  name: busybox
  namespace: default
  resourceVersion: "103042"
  uid: e2364b88-6249-4a81-bafd-92d12c0a26a3
spec:
  containers:
  - command:
    - sh
    - -c
    - sleep 1800
    image: busybox:1.36.1
    imagePullPolicy: IfNotPresent
    name: busybox
    resources:
      limits:
        cpu: 500m
        memory: 128Mi
      requests:
        cpu: 500m
        memory: 128Mi
    terminationMessa

In [30]:
!kubectl delete pod --force busybox

pod "busybox" force deleted


In [31]:
!kubectl apply -f .

pod/busybox created


In [33]:
!kubectl exec busybox -- sh -c "ls /etc/db/dev"

password
username


In [34]:
!kubectl exec busybox -- sh -c "cat /etc/db/dev/password"

db_pass


In [37]:
!kubectl delete --force -f .

pod "busybox" force deleted


In [38]:
!kubectl delete secret db-creds

secret "db-creds" deleted
